In [5]:
import opendota
import time
import sqlite3

In [2]:
conn = sqlite3.connect('dota_matches.db')
cursor = conn.cursor()
cursor.execute('''
    CREATE TABLE IF NOT EXISTS match_moves (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        choice TEXT,
        hero_id INTEGER,
        team TEXT,
        move_order INTEGER,
        match_id INTEGER,
        winner TEXT
    )
''')
conn.commit()

In [ ]:
# under the picks_bans key

# match id
# type - pick/ban
# order - 0,1,2,3 etc for all picks or bans
# hero id - hero id
# team - radiant/dire
master_max_retries = 2500
current_retries_cnt = 0
try:
    current_min_matchid = 8190370913
    for _ in range(1000):
        match_infos = client.get_pro_matches(match_id=current_min_matchid)
        time.sleep(0.2)
        for mi in match_infos:
            id = mi['match_id']
            winner = 'radiant' if mi['radiant_win'] else 'dire'

            # Retry logic for client.get_match up to 5 times
            match = None
            for _ in range(5):
                current_retries_cnt += 1
                match = client.get_match(str(id))
                if match is not None:
                    break
                time.sleep(0.5) 

            if current_retries_cnt > master_max_retries:
                raise Exception("Woah doggy, why so many internal errors, lets not pay a gazillion dollars to opendota")

            # If still no match, continue to next iteration
            if match is None:
                continue

            # not captains draft? 
            if "picks_bans" not in match:
                continue

            draft_order = match["picks_bans"]

            for move in draft_order:
                choice = "pick" if move["is_pick"] else "ban"
                hero_id = move["hero_id"]
                team = 'radiant' if move["team"] == 0 else 'dire'
                order = move["order"]

                cursor.execute('''
                    INSERT INTO match_moves (choice, hero_id, team, move_order, match_id, winner)
                    VALUES (?, ?, ?, ?, ?, ?)
                ''', (choice, hero_id, team, order, id, winner))
                conn.commit()
                
            time.sleep(0.2)
        current_min_matchid = mi["match_id"]

    conn.close()
except Exception as e:
    print(e)
finally:
    conn.close()

Could not fetch '/matches/8098261383' (Internal Server Error).


Expecting value: line 1 column 1 (char 0)


In [3]:
import pandas as pd
con = sqlite3.connect("dota_matches.db")
df = pd.read_sql_query("SELECT * from match_moves", con)
print(df.tail(50))
con.close()

            id choice  hero_id     team  move_order    match_id   winner
847682  847683   pick      109  radiant          22  7500239759     dire
847683  847684   pick       19     dire          23  7500239759     dire
847684  847685    ban      102  radiant           0  7500225480  radiant
847685  847686    ban      112     dire           1  7500225480  radiant
847686  847687    ban       37     dire           2  7500225480  radiant
847687  847688    ban       72  radiant           3  7500225480  radiant
847688  847689    ban      137     dire           4  7500225480  radiant
847689  847690    ban       86     dire           5  7500225480  radiant
847690  847691    ban       41  radiant           6  7500225480  radiant
847691  847692   pick       91  radiant           7  7500225480  radiant
847692  847693   pick       58     dire           8  7500225480  radiant
847693  847694    ban       42  radiant           9  7500225480  radiant
847694  847695    ban       11  radiant          10

In [4]:
len(df['match_id'].unique())

35395